# Input / Output

## PyVista

This notebook requires `NumPy`, `PyVista` and `tetgen` to be installed.

We are going to build a compound mesh, that includes 1d, 2d and 3d cells alike. As a source, use and example from `pyVista`.

In [1]:
from pyvista import examples
from dewloosh.mesh import PolyData
pv_bunny = examples.download_bunny_coarse()


A PolyData instance can be generated using the `from_pv` method of the class:

In [2]:
bunny_T3 = PolyData.from_pv(pv_bunny)


In [3]:
from copy import deepcopy
clone_T3 = deepcopy(bunny_T3)


Now turn the surface triangulation into a body tetrahedralization using `tetgen`, which actually understands `pyVista` objects pretty good.

In [4]:
# tetrahedralize
import tetgen, numpy as np
tet = tetgen.TetGen(pv_bunny)
tet.tetrahedralize(order=1, mindihedral=10, minratio=1.1, quality=True)
coords = np.array(tet.grid.points).astype(float)
topo = tet.grid.cells_dict[10].astype(int)  # VTK_TETRA --> 10

# create dewloosh object
from dewloosh.mesh.cells import TET4
from dewloosh.mesh.space import CartesianFrame
frame = CartesianFrame(dim=3)
bunny_TET4 = PolyData(coords=coords, topo=topo, celltype=TET4, frame=frame)
bunny_TET4.plot()

# split to two and transform the ones to the left to L2 lines
from dewloosh.mesh.cells import L2
from dewloosh.mesh.topo.tr import TET4_to_L2
c = bunny_TET4.centers()
xmin, xmax = coords[:, 2].min(), coords[:, 2].max()
left = np.where(c[:, 2] < ((xmin + xmax)/2))[0]
right = np.where(c[:, 2] >= ((xmin + xmax)/2))[0]
_, topo_L = TET4_to_L2(coords, topo[left])
topo_R = topo[right]

f:\GitHub\dewloosh-mesh\.venv\lib\site-packages\pyvista\plotting\plotting.py:5001: UserWarning: Not within a jupyter notebook environment.
Ignoring ``jupyter_backend``.
  warnings.warn(


In [5]:
pd = PolyData(coords=coords, frame=frame)
pd['L2'] = PolyData(topo=topo_L, celltype=L2, frame=frame)
pd['TET4'] = PolyData(topo=topo_R, celltype=TET4, frame=frame)

In [6]:
pd['L2']['_pyvista_'] = dict(color='red', line_width=0.5, style='wireframe', opacity=0.99)

In [7]:
pd['L2'].config['pyvista', 'plot'] = dict(a=1, b=2)

In [8]:
pd['L2'].config['pyvista', 'plot']

{'a': 1, 'b': 2}

In [9]:
pd['L2'].config['pyvista', 'plot']

{'a': 1, 'b': 2}

In [10]:
pd.plot()

## Gmsh

In [11]:
from math import pi
import pygmsh

In [12]:
with pygmsh.geo.Geometry() as geom:
    print('geometry type : ', type(geom))
    poly = geom.add_polygon(
        [
            [0.0, 200, 0.0],
            [0.0, 1200, 0.0],
            [0.0, 1200, 1000],
        ],
        mesh_size = 100,
    )
    geom.revolve(poly, [0.0, 0.0, 1.0], [0.0, 0.0, 0.0], 0.8 * pi)
    mesh = geom.generate_mesh()
    print('mesh type : ', type(mesh))
    edict = mesh.cells_dict
    points = mesh.points
    print(points.shape)
    cells = edict['tetra']
    print(cells.shape)

geometry type :  <class 'pygmsh.geo.geometry.Geometry'>
mesh type :  <class 'meshio._mesh.Mesh'>
(1556, 3)
(6008, 4)
